<a href="https://colab.research.google.com/github/castlechoi/studyingDL/blob/main/NLP/Word2Vec_CBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preprocessing

## Download corpus

In [82]:
import torch
import torch.optim as optim
import torch.nn as nn

import nltk
nltk.download("book", quiet = True)

True

In [83]:
# nltk.book에 저장된 다양한 corpus
from nltk.book import *
nltk.book.texts()

text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [84]:
# tokenize 모두 완료 되어 있음
ex_book = nltk.book.text1[:100]

## Stop-words 제거

In [85]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

# stop-words에 특수기호 추가
stopwords  = stopwords + ['.',',','\'','!','?','\"','[',']','(',')','*','I',':',';','-','."','--','<','>']

print(f'Stop-words의 개수 : {len(stopwords)}')

Stop-words의 개수 : 198


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [86]:
# 불용어 빠진 것을 확인
ex_book_no_stopwords = [[t] for t in ex_book if t not in stopwords]
print(f'불용어 제외한 후 문장의 길이 : {len(ex_book_no_stopwords)}')

불용어 제외한 후 문장의 길이 : 50


## Data preprocessing hyperparameter


In [87]:
min_count = 2
window = 2

## One-hot Encoding

In [88]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

In [89]:
# 토큰 집합 추출 ( 등장횟수가 2이하면 토큰화 안함)
cut_off = 0

tokens = pd.Series(ex_book_no_stopwords).value_counts()
for i in range(len(tokens)):
  if tokens[i] == min_count-1:
    cut_off = i
    break
tokens = tokens[:cut_off].index.tolist()
print(f'Token의 개수 : {len(tokens)}')

Token의 개수 : 4


In [90]:
# token에 없는 데이터 모두 <unk>로 변경
ex_book_process = [t if t in tokens else ['<unk>'] for t in ex_book_no_stopwords]

In [91]:
# One-Hot Encoding
oe = OneHotEncoder()
document_matrix = oe.fit_transform(ex_book_process)
print(f'문서의 단의 개수 : {document_matrix.shape[0]}')
print(f'One-Hot vector의 크기 : {document_matrix.shape[1]}')

문서의 단의 개수 : 50
One-Hot vector의 크기 : 5


## GPU 설정

In [92]:
#GPU 체크
is_cuda = torch.cuda.is_available()
if is_cuda:
  device = torch.device("cuda")
  print("GPU is available")
else:
  device = torch.device("cpu")
  print("CPU is availalbe")

GPU is available


# CBOW Low-level

## Train data preprocessing 

In [93]:
# train_x에 CBOW의 input으로 들어가는 4개의 벡터
train_x = []
train_y = []
for i in range(document_matrix.shape[0] - (window * 2)):
  neighbor = []
  neighbor.append(document_matrix[i].toarray())
  neighbor.append(document_matrix[i+1].toarray())
  neighbor.append(document_matrix[i+3].toarray())
  neighbor.append(document_matrix[i+4].toarray())

  train_x.append(neighbor)
  train_y.append(document_matrix[i+2].toarray())

In [94]:
train_x_tensor = torch.FloatTensor(train_x).view(-1,4,document_matrix.shape[1]).to(device)
train_y_tensor = torch.FloatTensor(train_y).view(-1,document_matrix.shape[1]).to(device)

print(f'train_x의 shape : {train_x_tensor.shape}') # 단어 개수 * 4 * one_hot
print(f'train_y의 shape : {train_y_tensor.shape}') # 단어 개수 * one_hot

train_x의 shape : torch.Size([46, 4, 5])
train_y의 shape : torch.Size([46, 5])


## Define Model

In [95]:
# Hyperparameter
num_epochs = 5000
lr = 0.001
emb_vector_size = 2

In [96]:
# Define weights without bias
W = torch.randn(document_matrix.shape[1],emb_vector_size).to(device).requires_grad_()
W_prime = torch.randn(emb_vector_size,document_matrix.shape[1]).to(device).requires_grad_()

In [97]:
# Define optimizer and loss
CBOW_optimizer = optim.Adam([W], lr = lr)
CBOW_optimizer_p = optim.Adam([W_prime], lr = 0.001)
criterion = nn.CrossEntropyLoss()

## Train the model

In [98]:
for i in range(num_epochs+1):
  # Input : 4 neighbor vector
  y_pred = train_x_tensor @ W
  y_pred = torch.mean(y_pred, dim = 1)
  
  # Input : Embedding vector
  # Output : predict one-hot vector
  y_pred = y_pred @ W_prime
  y_pred = y_pred.softmax(dim = 1)

  # compute loss
  loss = criterion(y_pred , train_y_tensor)
  
  # initiate optimizer
  CBOW_optimizer.zero_grad()
  CBOW_optimizer_p.zero_grad()
  # backpropagation
  loss.backward()
  CBOW_optimizer.step()
  CBOW_optimizer_p.step()

  if i % 500 == 0:
    print(f'epoch {i} : {loss.item()}')


epoch 0 : 1.797547698020935
epoch 500 : 1.617470145225525
epoch 1000 : 1.1250122785568237
epoch 1500 : 1.088291883468628
epoch 2000 : 1.0827877521514893
epoch 2500 : 1.0809146165847778
epoch 3000 : 1.0800526142120361
epoch 3500 : 1.0795886516571045
epoch 4000 : 1.0793136358261108
epoch 4500 : 1.0791399478912354
epoch 5000 : 1.079025149345398


## Print lookuptable

In [99]:
print(W)
print(W_prime)

tensor([[ 1.7064, -0.6959],
        [ 1.2750, -1.2640],
        [ 1.1298, -1.7958],
        [ 0.9281, -1.0241],
        [ 2.1942, -0.9534]], device='cuda:0', requires_grad=True)
tensor([[ 2.2417, -2.6511, -0.9970, -0.2650, -2.0126],
        [-2.1304, -0.4042,  2.6277,  3.2127,  0.7090]], device='cuda:0',
       requires_grad=True)


# CBOW Using nn.Linear

## Define Model

In [100]:
# Hyperparameter
num_epochs = 5000
lr = 0.001
emb_vector_size = 2

In [101]:
# Define model, optimizer and loss
CBOW_linear = nn.Linear(document_matrix.shape[1], emb_vector_size, bias = False).to(device)
CBOW_linear_p = nn.Linear(emb_vector_size,document_matrix.shape[1], bias = False).to(device)

CBOW_linear_optimizer = optim.Adam(CBOW_linear.parameters(), lr = lr)
CBOW_linear_optimizer_p = optim.Adam(CBOW_linear_p.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()

## Train the model

In [102]:
for i in range(num_epochs+1):
  # Input : 4 neighbor vector
  y_pred = CBOW_linear(train_x_tensor)
  y_pred = torch.mean(y_pred, dim = 1)

  # Input : Embedding vector
  # Output : predict one-hot vector
  y_pred = CBOW_linear_p(y_pred)
  y_pred = y_pred.softmax(dim = 1)
  
  # compute loss
  loss = criterion(y_pred , train_y_tensor)
  
  # initiate optimizer
  CBOW_linear_optimizer.zero_grad()
  CBOW_linear_optimizer_p.zero_grad()
  # backpropagation
  loss.backward()
  CBOW_linear_optimizer.step()
  CBOW_linear_optimizer_p.step()

  if i % 500 == 0:
    print(f'epoch {i} : {loss.item()}')

epoch 0 : 1.6192408800125122
epoch 500 : 1.2263877391815186
epoch 1000 : 1.0951097011566162
epoch 1500 : 1.0844076871871948
epoch 2000 : 1.08151113986969
epoch 2500 : 1.080324649810791
epoch 3000 : 1.079729437828064
epoch 3500 : 1.0793930292129517
epoch 4000 : 1.0791875123977661
epoch 4500 : 1.0790551900863647
epoch 5000 : 1.078966498374939


## Print lookuptable

In [103]:
for emb in CBOW_linear.parameters():
  print(emb)
for emb in CBOW_linear_p.parameters():
  print(emb)

Parameter containing:
tensor([[-1.6710, -1.4937, -1.3937, -1.7839, -1.9902],
        [ 1.2332,  0.9611,  1.1870,  0.8982,  0.9458]], device='cuda:0',
       requires_grad=True)
Parameter containing:
tensor([[-1.0023,  2.1965],
        [ 2.0194, -1.6008],
        [ 1.9349, -1.6525],
        [ 1.4033, -1.8451],
        [ 1.4956, -1.9645]], device='cuda:0', requires_grad=True)
